### e5-large-v2 embedding

In [3]:
import os

In [7]:
os.getcwd()
# os.chdir('..')

'/Users/franciscomeza/Desktop/ING/ING_MAT_COMP/9_semestre/Sistemas_recomendadores/Proyecto/ChessMatch-V2'

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

# =======================
# 1. CARGAR DATOS
# =======================

df = pd.read_csv("csvs/player_moves.csv")  # Reemplaza con la ruta a tu CSV
df["moves"] = df["moves"].str.replace("  ", " ", regex=False).str.strip()
#Eliminamos valores nulos
df = df.dropna(subset=["moves"]).copy()
df["moves"] = df["moves"].astype(str)
df[df["moves"].apply(lambda x: len(str(x).split())) <= 1].head()
df = df[df["moves"].apply(lambda x: len(str(x).split())) > 1]
df[df.duplicated(subset=["player", "moves"])]
df = df.drop_duplicates(subset=["player", "moves"], keep="first")


# =======================
# 2. CARGAR MODELO E5
# =======================

model = SentenceTransformer("intfloat/e5-large-v2")

# =======================
# 3. GENERAR EMBEDDINGS
# =======================

tqdm.pandas(desc="Generando embeddings")

df["embedding"] = df["moves"].progress_apply(
    lambda x: model.encode(x, normalize_embeddings=True)
)

# =======================
# 4. GUARDAR COMO PKL
# =======================

df.to_pickle("embeddings_e5_large_v2.pkl")

# =======================
# 5. GUARDAR COMO CSV
# =======================

# Expandimos el vector embedding en 1024 columnas
embedding_df = pd.DataFrame(df["embedding"].tolist())
embedding_df.columns = [f"dim_{i}" for i in range(embedding_df.shape[1])]

# Concatenamos con las columnas originales
final_df = pd.concat([df.drop(columns="embedding"), embedding_df], axis=1)

# Guardamos
final_df.to_csv("embeddings_e5_large_v2.csv", index=False)
print("✅ Archivos .pkl y .csv generados.")

Generando embeddings:   0%|          | 114/99804 [00:23<4:32:18,  6.10it/s]